In [74]:
import pandas as pd

# import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn_pandas import DataFrameMapper

from keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

import numpy as np
import json

from sklearn.metrics import mean_squared_error




# Read the CSV and Perform Basic Data Cleaning

In [75]:
df = pd.read_csv("Resources/03_new_df.csv")
df.describe()

,Opening,Theaters,Budget,Runtime,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Writer_Yvette Kaplan,Writer_Zach Helm,Writer_Zach Shields,Writer_Zack Snyder,Writer_Zack Stentz,Writer_Zak Penn,Writer_Zane Smith,Writer_Zeke Richardson,Writer_Álvaro Rodríguez,Writer_Éric Toledano
count,1.907000e+03,1907.000000,1.907000e+03,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,...,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000,1907.000000
mean,2.792583e+07,2913.979549,6.206534e+07,110.750393,0.359203,0.280545,0.093865,0.409544,0.149449,0.004719,...,0.000524,0.000524,0.000524,0.002622,0.001573,0.004719,0.000524,0.000524,0.000524,0.000524
std,2.729695e+07,702.092481,5.495635e+07,18.674676,0.479893,0.449384,0.291717,0.491879,0.356624,0.068554,...,0.022899,0.022899,0.022899,0.051151,0.039642,0.068554,0.022899,0.022899,0.022899,0.022899
min,1.000198e+07,629.000000,0.000000e+00,26.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.339860e+07,2503.000000,2.400000e+07,97.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.896668e+07,2950.000000,4.600000e+07,108.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.053719e+07,3389.500000,8.000000e+07,122.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,3.571150e+08,4662.000000,5.000000e+08,201.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [76]:
df.head()

,Studio,Opening,Theaters,Date,Budget,Runtime,Rating,Genre_Action,Genre_Adventure,Genre_Animation,...,Writer_Yvette Kaplan,Writer_Zach Helm,Writer_Zach Shields,Writer_Zack Snyder,Writer_Zack Stentz,Writer_Zak Penn,Writer_Zane Smith,Writer_Zeke Richardson,Writer_Álvaro Rodríguez,Writer_Éric Toledano
0,BV,357115007,4662,2019-04-26,356000000,181.0,PG-13,1,1,0,...,0,0,0,0,0,0,0,0,0,0
1,BV,257698183,4474,2018-04-27,300000000,149.0,PG-13,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,BV,247966675,4134,2015-12-18,245000000,136.0,PG-13,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3,BV,220009584,4232,2017-12-15,200000000,152.0,PG-13,1,1,0,...,0,0,0,0,0,0,0,0,0,0
4,Uni.,208806270,4274,2015-06-12,150000000,124.0,PG-13,1,1,0,...,0,0,0,0,0,0,0,0,0,0


# Change Date

In [77]:
date_column = df["Date"].str.split("-", n = 2, expand = True) 
year_col= date_column[0] 
month_col= date_column[1] 
day_col= date_column[2] 

In [78]:
# one_hot_year = OneHotEncoder(dtype=np.int, categories="auto", sparse=False).fit_transform(year_col.values.reshape(-1,1))
# one_hot_year = pd.Series(list(one_hot_year))
# # Create DataFrame 
# new_year_df = pd.DataFrame(one_hot_year, columns=["Year"]) 

In [79]:
# one_hot_month = OneHotEncoder(dtype=np.int, categories="auto", sparse=False).fit_transform(month_col.values.reshape(-1,1))
# one_hot_month = pd.Series(list(one_hot_month))
# new_month_df = pd.DataFrame(one_hot_month, columns=["Month"]) 

In [80]:
# one_hot_day = OneHotEncoder(dtype=np.int, categories="auto", sparse=False).fit_transform(day_col.values.reshape(-1,1))
# one_hot_day = pd.Series(list(one_hot_day))
# new_day_df = pd.DataFrame(one_hot_day, columns=["Day"]) 

In [81]:
year_col.describe()

count     1907
unique      40
top       2012
freq        89
Name: 0, dtype: object

In [82]:
label_encoder = LabelEncoder()
label_encoder.fit(year_col)
encoded_year = label_encoder.transform(year_col)

In [83]:
encoded_year

array([39, 38, 35, ..., 26, 34, 32])

In [84]:
one_hot_year = to_categorical(encoded_year)
one_hot_year

array([[0., 0., 0., ..., 0., 0., 1.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [85]:
one_hot_year.shape

(1907, 40)

In [86]:
one_hot_month = to_categorical(month_col)
one_hot_month

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 1.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [87]:
one_hot_month.shape

(1907, 13)

In [88]:
one_hot_day = to_categorical(day_col)
one_hot_day

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 1., 0., 0.]], dtype=float32)

In [89]:
one_hot_day.shape

(1907, 32)

In [90]:
year_df = pd.DataFrame(one_hot_year).add_prefix('Year_')

In [91]:
month_df = pd.DataFrame(one_hot_month).add_prefix('Month_')

In [92]:
day_df = pd.DataFrame(one_hot_day).add_prefix('Day_')

In [93]:
# date_df = pd.concat([year_df, month_df, day_df], axis=1)
# trying it without YEAR
date_df = pd.concat([month_df, day_df], axis=1)
date_df.shape

(1907, 45)

In [94]:
df = df.drop(columns=["Date"])
df = pd.get_dummies(df)
df.head()

,Opening,Theaters,Budget,Runtime,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Studio_Uni.,Studio_W/Dim.,Studio_WB,Studio_WB (NL),Studio_Wein.,Rating_G,Rating_NR,Rating_PG,Rating_PG-13,Rating_R
0,357115007,4662,356000000,181.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,257698183,4474,300000000,149.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,247966675,4134,245000000,136.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,220009584,4232,200000000,152.0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
4,208806270,4274,150000000,124.0,1,1,0,0,0,0,...,1,0,0,0,0,0,0,0,1,0


In [95]:
df = pd.concat([df, date_df], axis=1)
df.head()

,Opening,Theaters,Budget,Runtime,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,...,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31
0,357115007,4662,356000000,181.0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,257698183,4474,300000000,149.0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,247966675,4134,245000000,136.0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,220009584,4232,200000000,152.0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,208806270,4274,150000000,124.0,1,1,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


# Create a Train Test Split
Use `Opening` for the y values

In [96]:
target = df["Opening"]
data = df.drop("Opening", axis=1)
feature_names = data.columns
data.head()

,Theaters,Budget,Runtime,Genre_Action,Genre_Adventure,Genre_Animation,Genre_Comedy,Genre_Crime,Genre_Documentary,Genre_Drama,...,Day_22,Day_23,Day_24,Day_25,Day_26,Day_27,Day_28,Day_29,Day_30,Day_31
0,4662,356000000,181.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,4474,300000000,149.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
2,4134,245000000,136.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,4232,200000000,152.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4274,150000000,124.0,1,1,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [97]:
X_train, X_test, y_train, y_test = train_test_split(data, target, random_state=42)

In [98]:
X_train.shape

(1430, 16314)

In [99]:
y_train.shape

(1430,)

# Pre-processing

Scale the data using the MinMaxScaler

In [100]:
X_minmax = MinMaxScaler().fit(X_train)
# y_train.values.reshape(-1, 1)
y_minmax = MinMaxScaler().fit(y_train.values.reshape(-1, 1))

In [101]:
X_train_scaled = X_minmax.transform(X_train)
X_test_scaled = X_minmax.transform(X_test)
y_train_scaled = y_minmax.transform(y_train.values.reshape(-1, 1))
y_test_scaled = y_minmax.transform(y_test.values.reshape(-1, 1))

In [102]:
# X_mapper = DataFrameMapper([(['Theaters', 'Budget', 'Runtime'], MinMaxScaler())], 
#                           default=None).fit(X_train)

In [103]:
# X_train_scaled = X_mapper.transform(X_train)
# X_test_scaled = X_mapper.transform(X_test)

In [104]:
X_train_scaled.shape

(1430, 16314)

In [ ]:
X_train_scaled

In [ ]:
y_train.shape

# Try simple linear regression

In [49]:
# Import dependencies
from sklearn.linear_model import LinearRegression

# Create a linear model
model = LinearRegression()

# Fit (Train) our model to the data
model.fit(X_train_scaled, y_train_scaled)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [50]:
predictions = model.predict(X_test_scaled)

In [51]:
# Overall Score for the model
model.score(X_train_scaled, y_train_scaled)

1.0

In [52]:
model.score(X_test_scaled, y_test_scaled)

-0.32788325379749295

In [53]:
MSE = mean_squared_error(y_test_scaled, predictions)
r2 = model.score(X_test_scaled, y_test_scaled)
### END SOLUTION

print(f"MSE: {MSE}, R2: {r2}")

MSE: 0.004961736201694682, R2: -0.32788325379749295


# LASSO model

In [54]:
# LASSO model
# Note: Use an alpha of .01 when creating the model for this activity
from sklearn.linear_model import Lasso

### BEGIN SOLUTION
lasso = Lasso(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = lasso.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = lasso.score(X_test_scaled, y_test_scaled)
### END SOLUTION

print(f"MSE: {MSE}, R2: {r2}")

MSE: 0.0035687697266425815, R2: 0.04490900684128174


In [55]:
lasso.score(X_train_scaled, y_train_scaled)

0.03869454088908797

In [56]:
lasso.score(X_test_scaled, y_test_scaled)

0.04490900684128174

# Ridge model

In [57]:
# Ridge model
# Note: Use an alpha of .01 when creating the model for this activity
from sklearn.linear_model import Ridge

### BEGIN SOLUTION
ridge = Ridge(alpha=.01).fit(X_train_scaled, y_train_scaled)

predictions = ridge.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = ridge.score(X_test_scaled, y_test_scaled)
### END SOLUTION

print(f"MSE: {MSE}, R2: {r2}")

MSE: 0.0044956111888195425, R2: -0.20313667848346606


In [58]:
ridge.score(X_train_scaled, y_train_scaled)

0.9999659567955442

In [59]:
ridge.score(X_test_scaled, y_test_scaled)

-0.20313667848346606

# ElasticNet model

In [60]:
# ElasticNet model
# Note: Use an alpha of .01 when creating the model for this activity
from sklearn.linear_model import ElasticNet

### BEGIN SOLUTION
elasticnet = ElasticNet(alpha=.01).fit(X_train_scaled, y_train_scaled)
model.score(X_train_scaled, y_train_scaled)

model.score(X_test_scaled, y_test_scaled)
predictions = elasticnet.predict(X_test_scaled)

MSE = mean_squared_error(y_test_scaled, predictions)
r2 = elasticnet.score(X_test_scaled, y_test_scaled)
### END SOLUTION

print(f"MSE: {MSE}, R2: {r2}")

MSE: 0.002621397527550023, R2: 0.2984492248517925


In [61]:
elasticnet.score(X_train_scaled, y_train_scaled)

0.2336431355840808

In [62]:
elasticnet.score(X_test_scaled, y_test_scaled)

0.2984492248517925

# Try Neural Net: Define the architecture

In [112]:
nn_model = Sequential()

In [113]:
nn_model.add(Dense(units=1500, input_dim=16314, activation='relu'))
nn_model.add(Dense(units=800, activation='relu'))
nn_model.add(Dense(units=300, activation='relu'))
nn_model.add(Dense(units=1, activation='linear'))
nn_model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_23 (Dense)             (None, 1500)              24472500  
_________________________________________________________________
dense_24 (Dense)             (None, 800)               1200800   
_________________________________________________________________
dense_25 (Dense)             (None, 300)               240300    
_________________________________________________________________
dense_26 (Dense)             (None, 1)                 301       
Total params: 25,913,901
Trainable params: 25,913,901
Non-trainable params: 0
_________________________________________________________________


In [114]:
nn_model.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])

In [115]:
nn_model.fit(
    X_train_scaled,
    y_train_scaled,
    epochs=300,
    shuffle=False,
    verbose=2
)

Train on 1430 samples
Epoch 1/300
1430/1430 - 5s - loss: 0.0075 - mse: 0.0075 - mae: 0.0531
Epoch 2/300
1430/1430 - 5s - loss: 0.0032 - mse: 0.0032 - mae: 0.0324
Epoch 3/300
1430/1430 - 5s - loss: 0.0027 - mse: 0.0027 - mae: 0.0295
Epoch 4/300
1430/1430 - 5s - loss: 0.0025 - mse: 0.0025 - mae: 0.0256
Epoch 5/300
1430/1430 - 5s - loss: 0.0039 - mse: 0.0039 - mae: 0.0252
Epoch 6/300
1430/1430 - 5s - loss: 0.0033 - mse: 0.0033 - mae: 0.0295
Epoch 7/300
1430/1430 - 5s - loss: 0.0047 - mse: 0.0047 - mae: 0.0262
Epoch 8/300
1430/1430 - 5s - loss: 0.0043 - mse: 0.0043 - mae: 0.0237
Epoch 9/300
1430/1430 - 5s - loss: 0.0028 - mse: 0.0028 - mae: 0.0225
Epoch 10/300
1430/1430 - 5s - loss: 0.0031 - mse: 0.0031 - mae: 0.0248
Epoch 11/300
1430/1430 - 5s - loss: 0.0023 - mse: 0.0023 - mae: 0.0228
Epoch 12/300
1430/1430 - 5s - loss: 0.0020 - mse: 0.0020 - mae: 0.0202
Epoch 13/300
1430/1430 - 5s - loss: 0.0012 - mse: 0.0012 - mae: 0.0193
Epoch 14/300
1430/1430 - 5s - loss: 5.5235e-04 - mse: 5.5235e-04

Epoch 107/300
1430/1430 - 5s - loss: 6.8808e-05 - mse: 6.8808e-05 - mae: 0.0043
Epoch 108/300
1430/1430 - 5s - loss: 1.0121e-04 - mse: 1.0121e-04 - mae: 0.0050
Epoch 109/300
1430/1430 - 5s - loss: 5.0575e-05 - mse: 5.0575e-05 - mae: 0.0031
Epoch 110/300
1430/1430 - 5s - loss: 1.1894e-04 - mse: 1.1894e-04 - mae: 0.0037
Epoch 111/300
1430/1430 - 5s - loss: 1.0051e-04 - mse: 1.0051e-04 - mae: 0.0040
Epoch 112/300
1430/1430 - 5s - loss: 1.7261e-04 - mse: 1.7261e-04 - mae: 0.0063
Epoch 113/300
1430/1430 - 5s - loss: 9.2237e-05 - mse: 9.2237e-05 - mae: 0.0052
Epoch 114/300
1430/1430 - 5s - loss: 2.2118e-04 - mse: 2.2118e-04 - mae: 0.0074
Epoch 115/300
1430/1430 - 5s - loss: 1.5792e-04 - mse: 1.5792e-04 - mae: 0.0057
Epoch 116/300
1430/1430 - 5s - loss: 2.6375e-04 - mse: 2.6375e-04 - mae: 0.0062
Epoch 117/300
1430/1430 - 5s - loss: 1.4126e-04 - mse: 1.4126e-04 - mae: 0.0048
Epoch 118/300
1430/1430 - 5s - loss: 2.9423e-04 - mse: 2.9423e-04 - mae: 0.0070
Epoch 119/300
1430/1430 - 5s - loss: 2.4

Epoch 210/300
1430/1430 - 5s - loss: 2.5918e-05 - mse: 2.5918e-05 - mae: 0.0040
Epoch 211/300
1430/1430 - 5s - loss: 2.2459e-05 - mse: 2.2459e-05 - mae: 0.0035
Epoch 212/300
1430/1430 - 5s - loss: 1.4746e-05 - mse: 1.4746e-05 - mae: 0.0027
Epoch 213/300
1430/1430 - 5s - loss: 1.1771e-05 - mse: 1.1771e-05 - mae: 0.0022
Epoch 214/300
1430/1430 - 5s - loss: 1.0116e-05 - mse: 1.0116e-05 - mae: 0.0017
Epoch 215/300
1430/1430 - 5s - loss: 7.0849e-06 - mse: 7.0849e-06 - mae: 0.0013
Epoch 216/300
1430/1430 - 5s - loss: 1.4994e-05 - mse: 1.4994e-05 - mae: 0.0017
Epoch 217/300
1430/1430 - 5s - loss: 1.8061e-05 - mse: 1.8061e-05 - mae: 0.0020
Epoch 218/300
1430/1430 - 5s - loss: 1.8597e-05 - mse: 1.8597e-05 - mae: 0.0024
Epoch 219/300
1430/1430 - 5s - loss: 3.6949e-05 - mse: 3.6949e-05 - mae: 0.0033
Epoch 220/300
1430/1430 - 5s - loss: 3.7782e-05 - mse: 3.7782e-05 - mae: 0.0035
Epoch 221/300
1430/1430 - 5s - loss: 3.6803e-05 - mse: 3.6803e-05 - mae: 0.0040
Epoch 222/300
1430/1430 - 5s - loss: 9.7

In [118]:
nn_model.evaluate(
    X_test_scaled, y_test_scaled, verbose=2)

477/477 - 0s - loss: 0.0025 - mse: 0.0025 - mae: 0.0320


[0.0024977298175224904, 0.0024977298, 0.031986564]